In [1]:
import sys
import os
import csv
import random
import numpy as np
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from force_utils import database, save_history

Using TensorFlow backend.


In [2]:
# Make force_dic = {image_file_name : force_value}
movie_file = 'dataset/movie.mp4'
csv_file = 'dataset/forceinfo.csv'
dataset = database(movie_file, csv_file)
force = dataset.make_tension()
frame_id, frames = dataset.make_inputs()
force_dic = {}
for i in range(len(frames)):
    force_dic[frames[i]] = force[i]

# Split train and validation
keys = list(force_dic.keys())
train_num = int(round(0.8 * len(keys)))
train_keys = keys[:train_num]
val_keys = keys[train_num:]
val_num = len(val_keys)

In [4]:
bottleneck_features_train = np.load('dataset/features/vgg16_train.npy')
bottleneck_features_validation = np.load('dataset/features/vgg16_val.npy')

## DATASET

In [5]:
train_data = bottleneck_features_train.copy()
validation_data = bottleneck_features_validation.copy()

In [6]:
x_train = train_data[:-112]
x_test = validation_data[:-28]

In [7]:
y_train = []
for image_file in train_keys:
    y_train.append(force_dic[image_file])
y_train = np.array(y_train)[:-82]

In [8]:
y_test = []
for image_file in val_keys:
    y_test.append(force_dic[image_file])
y_test = np.array(y_test)[:-20]

In [9]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(45200, 9, 9, 512) (45200,) (11300, 9, 9, 512) (11300,)


## LSTM DATASET

In [10]:
batch_size=32
epochs=100
timesteps = 20
train_samples = int(x_train.shape[0] / timesteps)
validation_samples = int(x_test.shape[0] / timesteps)
height, width, channel = x_train.shape[1], x_train.shape[2], x_train.shape[3]
print(train_samples, validation_samples, height, width, channel)

2260 565 9 9 512


In [11]:
x_train_lstm = x_train.reshape(train_samples, timesteps, height, width, channel)
x_test_lstm = x_test.reshape(validation_samples, timesteps, height, width, channel)
print(x_train_lstm.shape, x_test_lstm.shape)

(2260, 20, 9, 9, 512) (565, 20, 9, 9, 512)


In [12]:
y_train_lstm = y_train.reshape(train_samples, timesteps, 1)[:, -1]
y_test_lstm = y_test.reshape(validation_samples, timesteps, 1)[:, -1]
print(y_train_lstm.shape, y_test_lstm.shape)

(2260, 1) (565, 1)


## MODEL

In [13]:
input_tensor = Input(shape=(timesteps, height, width, channel))
x = TimeDistributed(Flatten(input_shape=(height, width, channel)))(input_tensor)
x = LSTM(512, return_sequences=False, dropout=0.5)(x)
x = Dense(256, activation='relu')(x)
prediction = Dense(1)(x)
model = Model(inputs=input_tensor, outputs=prediction)

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20, 9, 9, 512)     0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 41472)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               85985280  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 86,116,865
Trainable params: 86,116,865
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Optimizer
optim = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

# Callbacks
callbacks = [
    ModelCheckpoint('dataset/weights/lstm_mae_{epoch:02d}.h5', monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')]

# Compile
model.compile(optimizer=optim, loss='mean_absolute_error')

In [16]:
history = model.fit(
    x_train_lstm, y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(x_test_lstm, y_test_lstm),
    verbose=1)

save_history(history, 'dataset/history/lstm_mae.txt')

Train on 2260 samples, validate on 565 samples
Epoch 1/100
2260/2260 [==============================] - 26s 12ms/step - loss: 1.0354 - val_loss: 0.5334

Epoch 00001: val_loss improved from inf to 0.53343, saving model to dataset/weights/lstm_mae_01.h5
Epoch 2/100
2260/2260 [==============================] - 21s 9ms/step - loss: 0.8712 - val_loss: 0.5585

Epoch 00002: val_loss did not improve from 0.53343
Epoch 3/100
2260/2260 [==============================] - 21s 9ms/step - loss: 0.8418 - val_loss: 0.9042

Epoch 00003: val_loss did not improve from 0.53343
Epoch 4/100
2260/2260 [==============================] - 21s 9ms/step - loss: 0.8114 - val_loss: 0.6785

Epoch 00004: val_loss did not improve from 0.53343
Epoch 5/100
2260/2260 [==============================] - 21s 9ms/step - loss: 0.7961 - val_loss: 0.5498

Epoch 00005: val_loss did not improve from 0.53343
Epoch 6/100
2260/2260 [==============================] - 21s 9ms/step - loss: 0.7784 - val_loss: 0.5661

Epoch 00006: val_lo